<p><span style="font-size:36px"><strong>2 Data Processing</strong></span></p>

In [24]:
import pandas as pd
import geopandas as gpd
import json as js
import numpy as np
import topojson as tp
import datetime
import shapely
from shapely import wkt
from shapely.wkb import loads, dumps
from shapely.geometry import Polygon, LineString, Point

<p><span style="font-size:18px"><strong>2.1 Elektroladestationen JSON in DataFrame umwandeln</strong></span></p>

Quelle: https://opendata.swiss/de/dataset/ladestationen-fuer-elektroautos/resource/4d467a51-0bc9-48ce-aa2a-3d3bcaa7e7e9

In [38]:
#JSON wird geladen
f = open('../Data/lade.json')
data = js.load(f)

In [39]:
#JSON wird in DF umgewandelt
OperatorNames = []
cities = []
plz = [] 
coords = []
for name in data['EVSEData']:
    for add in name['EVSEDataRecord']:
        OperatorNames.append(name['OperatorName'])
        cities.append(add['Address']['City'])
        plz.append(add['Address']['PostalCode'])
        coords.append(add['GeoCoordinates']['Google'])
        
d = {'OperatorName':OperatorNames,'City':cities,'Plz':plz,'Coord':coords}
df = pd.DataFrame(data=d)

In [40]:
#Koordinaten werden für spätere Schritte in zwei Spalten gesplittet
df['Lat']= df.apply(lambda x: x['Coord'].split(' ')[0], axis=1)
df['Lon']= df.apply(lambda x: x['Coord'].split(' ')[1],axis =1)

<p><span style="font-size:18px"><strong>2.2 Gemeindegrenzen richtig formattieren</strong></span></p>

Die Gemeindegrenzen wurden in einem vorherigen Schritt (siehe DataCollection Notebook) heruntergeladen. Jetzt gilt es die Daten in ein gängiges Format umzuwandeln, mit welchem die Grenzen richtig ausgelesen werden können. In dieser Domäne bietet sich ein GeoDF von GeoPandas sehr gut an, da GeoPandas viele Funktionen zum handeln von geografischen Daten besitzt.

In [28]:
gemdf = pd.read_csv('../Data/GemeindeGrenzen.csv')

Da Geopandas eine spezielle Formatierung braucht um die GeoDaten(MultiPolygone) als solche zu erkennen und es uns nicht gelungen ist diese korrekt einzulesen, werden Sie hier manuell in die korrekte Formatierung gebracht.

In [29]:
polytpl = []
for place in gemdf.Polygon:
    lst1 = [] 
    place = js.loads(str(place))['coordinates']
    for polobj in place:
        lst2 = []    
        for xobj in polobj:
            lst3 = []    
            for coordinates in xobj:
                   
                container = ''
                counter = 0
                for latlan in coordinates:
                    if counter == 1:
                        container += ' '
                    container += str(latlan)
                    counter +=1  
                lst3.append(container)
            lst2.append(tuple(lst3))
        lst1.append(tuple(lst2))
    polytpl.append(tuple(lst1))
polytpl = tuple((polytpl))

In [30]:
#Formatierung wird abgespeichert
gemdf['FormatPolygon'] = polytpl

In [31]:
#Daten werden weiter korrekt formatiert
gemdf['geometry'] = gemdf.apply(lambda x: str('MULTIPOLYGON' + str(x['FormatPolygon'])), axis = 1 )

In [32]:
#Daten werden zu ende formatiert und werden eingelesen
geometry_objects = [shapely.wkt.loads(str(s).replace("'",'').replace("),)",'))').replace(")),)",')))')) for s in gemdf.geometry]

In [33]:
#Daten werden als GeoPanda DF abgespeichert
geodf = gpd.GeoDataFrame(gemdf[['BFSid','GemeindeLabel']], geometry=gpd.GeoSeries(geometry_objects))

In [34]:
#Kontrolle
geodf

,BFSid,GemeindeLabel,geometry
0,1,Aeugst am Albis,"MULTIPOLYGON (((8.50389 47.27308, 8.50344 47.2..."
1,2,Affoltern am Albis,"MULTIPOLYGON (((8.41466 47.29360, 8.41496 47.2..."
2,3,Bonstetten,"MULTIPOLYGON (((8.44132 47.31583, 8.44132 47.3..."
3,4,Hausen am Albis,"MULTIPOLYGON (((8.57437 47.21707, 8.57434 47.2..."
4,5,Hedingen,"MULTIPOLYGON (((8.43621 47.31520, 8.43632 47.3..."
...,...,...,...
2143,6806,Vendlincourt,"MULTIPOLYGON (((7.15253 47.43921, 7.15221 47.4..."
2144,6807,Basse-Allaine,"MULTIPOLYGON (((7.03694 47.49751, 7.03798 47.4..."
2145,6808,Clos du Doubs,"MULTIPOLYGON (((7.19091 47.34843, 7.19060 47.3..."
2146,6809,Haute-Ajoie,"MULTIPOLYGON (((6.95744 47.40529, 6.95836 47.4..."


<p><span style="font-size:18px"><strong>2.3 Anzahl Elektroladestationen pro Gemeinde bestimmen</strong></span></p>

In [35]:
#DF wird auf korrektheit kontrolliert
df[df['OperatorName']=='eCarUp'].sort_values(by='Lat',ascending=True).head(60)

,OperatorName,City,Plz,Coord,Lat,Lon
8463,eCarUp,-,-,-0.818855 -18.988678,-0.818855,-18.988678
9218,eCarUp,-,-,-0.818883 -18.971714,-0.818883,-18.971714
9610,eCarUp,WA 6056,Midvale,-31.892987 116.032476,-31.892987,116.032476
8495,eCarUp,Vientiane,-,17.968313 102.612609,17.968313,102.612609
8496,eCarUp,Vientiane,-,17.968313 102.612609,17.968313,102.612609
9065,eCarUp,-,-,27.421028 -43.266836,27.421028,-43.266836
7234,eCarUp,-,-,28.841073 -43.272991,28.841073,-43.272991
7431,eCarUp,-,-,29.44057 -42.885544,29.44057,-42.885544
8921,eCarUp,-,-,29.675077 -42.238439,29.675077,-42.238439
7405,eCarUp,-,-,30.604648 -41.190937,30.604648,-41.190937


Wie man sehen kann gibt es gewisse Betreiber mit fehlerhafte Ladestationendaten, welche nicht in der Schweiz liegen. Zum Glück stimmen alle Koordinaten, weswegen man diese mit den Gemeindegrenzen abgleichen und die korrekte Gemeinde zuweisen kann.

In [41]:
#Koordinaten werden auf die Gemeinden abgegliechen und Gemeinde herausgesucht
ladeort = []
for x in range (0,df.shape[0]):
    try:
        ladeort.append(geodf[geodf.contains(Point(df['Lon'].iloc[x],df['Lat'].iloc[x]))]['GemeindeLabel'].values[0])
    except:
        ladeort.append(str('Error'))

In [42]:
#Die korrekten Gemeindebezeichnungen werden hinzugefügt
df['GemeindeLoc'] = ladeort

In [43]:
#Überblick über die falschen Daten und 'problematische' Betreiber, welche fehlerhafte Daten haben
df[(df['GemeindeLoc']=='Error') & (df['OperatorName']!='eCarUp')]

,OperatorName,City,Plz,Coord,Lat,Lon,GemeindeLoc
0,EWD Elektrizitätswerk Davos AG,Davos,7260,46.81352 9.84591,46.81352,9.84591,Error
1,EWD Elektrizitätswerk Davos AG,Davos,7260,46.8135 9.84586,46.8135,9.84586,Error
2,EWD Elektrizitätswerk Davos AG,Davos,7270,46.80008 9.83121,46.80008,9.83121,Error
3,EWD Elektrizitätswerk Davos AG,Davos,7270,46.800129999999996 9.83124,46.800129999999996,9.83124,Error
4,EWD Elektrizitätswerk Davos AG,Davos,7270,46.79978 9.83104,46.79978,9.83104,Error
...,...,...,...,...,...,...,...
9852,Fastned,Suhr,5034,47.380548 8.109115,47.380548,8.109115,Error
9853,Fastned,Lenzburg,5600,47.392556 8.160158,47.392556,8.160158,Error
9854,Fastned,Lenzburg,5600,47.392556 8.160158,47.392556,8.160158,Error
9855,Fastned,Lenzburg,5600,47.392556 8.160158,47.392556,8.160158,Error


In [44]:
#Der DF wird mit den korrekten Gemeinde IDs vom Bund (BFS-ID) versehen
BFSidlist = []
for x in range(0,df.shape[0]):
    try:
        BFSidlist.append(gemdf[gemdf['GemeindeLabel'] == df.GemeindeLoc.iloc[x]].BFSid.values[0])
    except:
        BFSidlist.append('NAN')

In [45]:
#Die BFS-ID werden hinzugefügt
df['BFSid'] = BFSidlist

In [47]:
#Das DF wird gecleaned. Nicht vorhandene Gemeinden (z. bsp. welche nicht in der Schweiz liegen werden gedroppt)
df = df.drop(df[df['BFSid'] == 'NAN'].index, axis=0)
df = df.drop(columns=['City','Plz','Coord'], axis=1)

In [48]:
geodf = geodf.merge(df.groupby('BFSid').count()['OperatorName'],how='left',on='BFSid')

In [49]:
#Ladestationen pro Gemeinde werden gezählt
geodf['AnzahlLadestation'] = geodf['OperatorName']
geodf['AnzahlLadestation'] = geodf['AnzahlLadestation'].fillna(0)
geodf = geodf.set_index('BFSid')
geodf = geodf.drop(columns='OperatorName', axis =1)

In [ ]:
geodf.to_csv('../Data/LadestationProGemeinde.csv')

<p><span style="font-size:18px"><strong>2.4 Geodaten simplifizieren um Speicher/Leistung zu sparen </strong></span></p>

Da die Gemeindedaten sehr speicher intensiv sind und auf Streamlit mehr performance zum laden brauchen werden sie in den unteren Schritten vereinfacht. Die Vereinfachungsfunktion von GeoPandas vereinfacht leider nicht ohne dass Lücken zwischen den Gemeinden entstehen, so dass auf die topojson liabrary ausgewichen werden musste.

In [50]:
geodf_simple = tp.Topology(geodf, topology=True, toposimplify=0.001).to_gdf()

In [ ]:
geodf_simple.to_csv('../Data/GemeindeGrenzen_Vereinfacht.csv')

<p><span style="font-size:18px"><strong>2.5 Ladestation Timeseries vorbereiten</strong></span></p>

Für die TimeSeries in unserer StreamlitApp bereiten wir hier die Daten in einer sogenannten 'geschmolzenen' Tabelle vor.<br /> Quelle: https://opendata.swiss/de/dataset/kennzahlen-offentliche-ladeinfrastruktur-elektromobilitat/resource/d772c474-e890-4de6-b1bb-416d90e860a3

In [52]:
ladstat = pd.read_csv('../Data/charging-infrastructure_key-figures.csv')

In [53]:
#Überblick Kolumnen
ladstat.columns.values

array(['year', 'month', 'chargingPower_10kW_count',
       'chargingPower_21kW_count', 'chargingPower_42kW_count',
       'chargingPower_100kW_count', 'chargingPower_100pluskW_count',
       'chargingPower_AG_count', 'chargingPower_AG_sum',
       'chargingPower_AI_count', 'chargingPower_AI_sum',
       'chargingPower_AR_count', 'chargingPower_AR_sum',
       'chargingPower_BE_count', 'chargingPower_BE_sum',
       'chargingPower_BL_count', 'chargingPower_BL_sum',
       'chargingPower_BS_count', 'chargingPower_BS_sum',
       'chargingPower_CH_count', 'chargingPower_CH_sum',
       'chargingPower_FR_count', 'chargingPower_FR_sum',
       'chargingPower_GE_count', 'chargingPower_GE_sum',
       'chargingPower_GL_count', 'chargingPower_GL_sum',
       'chargingPower_GR_count', 'chargingPower_GR_sum',
       'chargingPower_JU_count', 'chargingPower_JU_sum',
       'chargingPower_LU_count', 'chargingPower_LU_sum',
       'chargingPower_NE_count', 'chargingPower_NE_sum',
       'chargingPo

In [54]:
#Alle Kolumnen welche 'stations' enthalten sind für uns interessant. 
#Deswegen erstellen wir eine Liste mit diesen Kolumnen, welche wir nutzen werden um den DF zu filtern.
subs = 'stations'
res = [i for i in ladstat.columns.values if subs in i]
res.append('year')
res.append('month')
res

In [56]:
#Das gefilterte DF wird 'geschmolzen'
melted_df = ladstat[res].melt(id_vars=['year','month'],var_name='Canton',value_name='Amount')

In [57]:
#DF wird gecleaned
melted_df['Canton'] = melted_df.apply(lambda x: x['Canton'].replace('stations_','').replace('_count',''),axis=1)
melted_df['day'] = 1
melted_df['datetime'] = pd.to_datetime(melted_df[['year','month','day']])
melted_df = melted_df.drop(columns =['year','month','day'],axis=1)

In [58]:
#Kontrolle
melted_df

,Canton,Amount,datetime
0,AG,362,2020-11-01
1,AG,368,2020-12-01
2,AG,393,2021-01-01
3,AG,435,2021-02-01
4,AG,446,2021-03-01
...,...,...,...
643,ZH,1102,2022-07-01
644,ZH,1151,2022-08-01
645,ZH,1086,2022-09-01
646,ZH,1090,2022-10-01


In [ ]:
melted_df.to_csv('../Data/Chargingstations_melted.csv')